In [24]:
import preprocess

reload(preprocess)

from preprocess import load_datasets, preprocess_mushrooms_datasets
from importlib import reload
import pandas as pd


In [21]:
train_dfs = load_datasets('datasetsCBR/mushroom/mushroom.fold.*.train.arff')
test_dfs = load_datasets('datasetsCBR/mushroom/mushroom.fold.*.test.arff')

train_dfs[0]

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,b'x',b's',b'w',b't',b'l',b'f',b'c',b'b',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'n',b'm',b'e'
1,b'k',b'y',b'e',b'f',b's',b'f',b'c',b'n',b'b',b't',...,b'p',b'w',b'p',b'w',b'o',b'e',b'w',b'v',b'd',b'p'
2,b'x',b'y',b'y',b't',b'a',b'f',b'c',b'b',b'n',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b's',b'm',b'e'
3,b'x',b'y',b'w',b't',b'p',b'f',b'c',b'n',b'k',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'v',b'u',b'p'
4,b'x',b'y',b'n',b'f',b'f',b'f',b'c',b'n',b'b',b't',...,b'w',b'p',b'p',b'w',b'o',b'e',b'w',b'v',b'l',b'p'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7308,b'x',b'y',b'n',b't',b'l',b'f',b'c',b'b',b'w',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b's',b'p',b'e'
7309,b'x',b'y',b'y',b'f',b'f',b'f',b'c',b'b',b'p',b'e',...,b'b',b'n',b'p',b'w',b'o',b'l',b'h',b'v',b'g',b'p'
7310,b'x',b's',b'w',b'f',b'c',b'f',b'w',b'n',b'n',b'e',...,b'w',b'w',b'p',b'w',b'o',b'p',b'k',b'v',b'd',b'p'
7311,b'f',b'y',b'n',b'f',b'f',b'f',b'c',b'n',b'b',b't',...,b'w',b'w',b'p',b'w',b'o',b'e',b'w',b'v',b'p',b'p'


In [25]:
for df in train_dfs:
    preprocess_mushrooms_datasets(df)


,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,5,2,8,1,3,1,0,0,4,0,...,7,7,0,2,1,4,2,2,3,0
1,3,3,2,0,7,1,0,1,0,1,...,6,7,0,2,1,0,7,4,0,1
2,5,3,9,1,0,1,0,0,5,0,...,7,7,0,2,1,4,2,3,3,0
3,5,3,8,1,6,1,0,1,4,0,...,7,7,0,2,1,4,2,4,5,1
4,5,3,4,0,2,1,0,1,0,1,...,7,6,0,2,1,0,7,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73111,5,3,4,1,5,1,0,0,9,1,...,6,7,0,2,1,4,3,4,0,0
73112,5,3,2,1,5,1,0,0,10,1,...,6,3,0,2,1,4,2,5,0,0
73113,3,2,2,0,7,1,0,1,0,1,...,6,6,0,2,1,0,7,4,2,1
73114,5,0,8,0,5,1,1,0,3,1,...,7,7,0,2,1,0,2,3,1,0
